In [1]:
# code for loading in data set from:
# https://pythonbasics.org/how-to-load-machine-learning-data-in-python/
import numpy as np
import copy
filename = 'Data/CS170_Small_Data__82.txt'
raw_data = open(filename, 'rt')
data = np.loadtxt(raw_data)
print(data.shape)
print(data)

(500, 7)
[[ 2.         -0.63742821 -0.3877098  ... -0.78912863 -1.0888035
   0.33726993]
 [ 2.         -0.10508566  1.5406805  ...  1.173381   -0.30325924
   1.0776392 ]
 [ 2.         -0.83025536  1.3045544  ...  0.06139414  0.17492874
  -1.4731397 ]
 ...
 [ 2.         -0.43517064  1.077989   ... -0.69252039 -0.97886516
  -0.53556053]
 [ 1.          0.29658063 -0.89554223 ...  0.48519201  0.51102218
   0.77067154]
 [ 1.          0.56477704  1.713623   ... -0.73334086 -1.5821675
   0.55436715]]


In [48]:
def LeaveOneOutCrossValid(data, current_set, feature_to_add):
    # Leave one out cross validation, returns the accuracy
    rows = data.shape[0]
    cols = data.shape[1]
    # zero columns
    #for feature in current_set:
    #    data[:,feature] = 0 # this is a bug
    #data[:,feature_to_add] = 0 # this is a bug
    for feature in range(1,cols):
        if (feature not in current_set) and (feature != feature_to_add):
            data[:,feature] = 0

    NumCorrect = 0
    for i in range(rows):
        ObjToClassify = data[i,1:]
        LabelObjToClassify = data[i,0]
        #print(f"Looping over i, at the {i} location")
        #print(f"The {i}'th object is in class {LabelObjToClassify}")
        NearestNeighborDistance = np.inf
        NearestNeighborLocation = np.inf
        for k in range(rows):
            if not k == i: # except self
                distance = np.sqrt(np.sum((ObjToClassify - data[k,1:]) ** 2))
                if distance < NearestNeighborDistance:
                    NearestNeighborDistance = distance
                    NearestNeighborLocation = k
                    NearestNeighborLabel = data[NearestNeighborLocation,0]
                    #print(f"Ask if {i} is nearest neighbor with {k}")
                    #print("Distance: ", distance)
        if LabelObjToClassify == NearestNeighborLabel:
            NumCorrect = NumCorrect + 1
    
    accuracy = NumCorrect / rows
        #print(f"Object {i} is class {LabelObjToClassify}")
        #print(f"Its nearest neighbor is {NearestNeighborLocation} which is class {NearestNeighborLabel}")
    return accuracy

def feature_search(data):
    CurrentSet = set()
    BestAcc = 0
    for i in range(1, data.shape[1]):
        #print(f"On the {i}th level of the search tree")
        FeatureToAddAtThisLevel = set()
        BestSoFarAccuracy = 0
        
        for k in range(1, data.shape[1]):
            if not CurrentSet.intersection(set({k})):
                #print(f"--Considering adding feature {k}")
                # to create columns of zeros without affecting the original array
                Data = copy.deepcopy(data)
                #accuracy = LeaveOneOutCrossValid(Data, CurrentSet, k+1)
                accuracy = LeaveOneOutCrossValid(Data, CurrentSet, k)
                #print(f"accuracy of feature {CurrentSet} + feature {k} = : {accuracy}")

                if accuracy > BestSoFarAccuracy:
                    BestSoFarAccuracy = accuracy
                    FeatureToAddAtThisLevel = k
                    BestAcc = accuracy
                    print(CurrentSet)
        CurrentSet.add(FeatureToAddAtThisLevel) 
        print(f"On level {i} i added feature {FeatureToAddAtThisLevel} to current set")
        print(BestAcc)
        #print(CurrentSet)

In [ ]:
#feature_search(data)
LeaveOneOutCrossValid(data,0,0)

In [49]:
#filename = 'Data/SampleData/CS170_SMALLtestdata__1.txt'
filename = 'Data/SampleData/Sue/CS170_Small_Data__96.txt'
Raw_data = open(filename, 'rt')
Data = np.loadtxt(Raw_data)
#print(Data.shape)
#print(Data)
feature_search(Data)
#LeaveOneOutCrossValid(Data,0,0)

set()
On level 1 i added feature 1 to current set
0.874
{1}
{1}
{1}
On level 2 i added feature 6 to current set
0.948
{1, 6}
{1, 6}
On level 3 i added feature 3 to current set
0.94
{1, 3, 6}
{1, 3, 6}
{1, 3, 6}
On level 4 i added feature 5 to current set
0.892
{1, 3, 5, 6}
{1, 3, 5, 6}
On level 5 i added feature 4 to current set
0.864
{1, 3, 4, 5, 6}
On level 6 i added feature 2 to current set
0.836
